In [1]:
from collections import defaultdict
from datetime import datetime
import dill
from itertools import permutations, combinations
import json
from operator import itemgetter
import os
import pickle
import random
import re
import time

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from adjustText import adjust_text
sns.set(style='ticks', font_scale=1.2)
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

import little_mallet_wrapper as lmw

In [2]:
# data_directory_path   = '/Volumes/Passport-1/data/birth-control'
# output_directory_path = '/Volumes/Passport-1/output/birth-control'

data_directory_path   = '/Users/maria/Documents/data/birth-control'
output_directory_path = '/Users/maria/Documents/output/birth-control'

<br><br>

# Load datasets (Reddit, WebMD, Twitter)

## Reddit

In [3]:
reddit_posts_df = pd.read_csv(data_directory_path + '/final-data/reddit_posts.csv')
reddit_comments_df = pd.read_csv(data_directory_path + '/final-data/reddit_comments.csv')

In [4]:
len(reddit_posts_df.index), len(reddit_comments_df.index)

(68958, 264912)

In [5]:
reddit_comments_df.sample(3)

,Unnamed: 0,id,parent_id,created_utc,text,tokens_text,text_type,year,month,source
105594,1374,e8etgh3,t3_9qzlw0,1.540447e+09,Take 800 mg ibuprofen an hour beforehand! Mayb...,take NUM mg ibuprofen hour beforehand maybe ob...,iud,2018,10,reddit-comments
234165,266,fsbsno2,t3_gthucf,1.590861e+09,Extending the pill break is the most risky thi...,extending pill break risky thing would use bac...,pill,2020,5,reddit-comments
110424,2950,ea20pac,t3_9yj647,1.542659e+09,"Just out of curiosity, does anyone know if you...",curiosity anyone know sedated procedure mirena...,iud,2018,11,reddit-comments


In [6]:
reddit_posts_df.sample(3)

,Unnamed: 0,id,created_utc,text,title,year,month,url,link_flair_text,tokens_text,text_type,source
12752,64,54iqss,1474860202,"I used my last patch about a month ago, and I'...",Better mood after going off patch/second-guess...,2016,9,https://www.reddit.com/r/birthcontrol/comments...,Experience,better mood going patch/second guessing nexpla...,implant,reddit-posts
10818,337,49q7ct,1457558035,Has anyone had a paraguard AND a mirena? Shoul...,Mirena insertion?,2016,3,https://www.reddit.com/r/birthcontrol/comments...,Experience,mirena insertion anyone paraguard mirena expec...,iud,reddit-posts
64967,396,igzz0n,1598452557,"Background info: I’m new to taytulla, on my se...",newbie pill question. is it ok to skip sugar p...,2020,8,https://www.reddit.com/r/birthcontrol/comments...,How to?,newbie pill question ok skip sugar pills backg...,pill,reddit-posts


## WebMD

In [7]:
webmd_df = pd.read_csv(data_directory_path + '/final-data/webmd.csv')

In [8]:
len(webmd_df.index)

18110

In [9]:
webmd_df.sample(3)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,id,date,year,text,name,title,source,text_type,num_tokens,tokens_text
727,727,772,772,w793,2008-12-08,2008,I only used this medication (28 count dosage) ...,aviane,NaN,webmd-reviews,pill,43,used medication NUM count dosage NUM NUM weeks...
8769,8769,9823,9823,w10042,2011-02-13,2011,"Ok, so I had the mirena for a while. At first...",mirena-device,NaN,webmd-reviews,iud,204,ok mirena first singing praises like NUM NUM y...
11364,11364,12440,12440,w12732,2013-04-19,2013,I have been on this for almost a year and it h...,nortrel-1-35-21,NaN,webmd-reviews,pill,49,almost year great complain makes difficult los...


## Twitter

In [10]:
twitter_posts_df = pd.read_csv(data_directory_path + '/final-data/twitter_posts.csv')
twitter_replies_df = pd.read_csv(data_directory_path + '/final-data/twitter_replies.csv')

In [11]:
len(twitter_posts_df.index), len(twitter_replies_df.index)

(499796, 211896)

In [12]:
twitter_posts_df.sample(3)

,Unnamed: 0,source,text,tokens_text,date,year,month,id,conversation_id,retweet_count,reply_count,like_count,quote_count,text_type,num_tokens
293000,364251,twitter-posts,#contraceptive pill weight loss bergfest onlin...,contraceptive pill weight loss bergfest online...,2016-12-17T03:15:36.000Z,2016,12,809960205924581376,809960205924581376,0,0,0,0,pill,7
185655,234817,twitter-posts,The IUD is a contraceptive rock star. Nice cas...,iud contraceptive rock star nice case iud http...,2011-10-07T18:30:00.000Z,2011,10,122378155007815681,122378155007815681,1,0,0,0,iud,10
466225,586256,twitter-posts,Birth control pill... for men. http://www.cnn....,birth control pill men http //www cnn com/NUM/...,2010-07-02T22:19:28.000Z,2010,7,17604462823,17604462823,1,0,0,0,pill,12


In [13]:
twitter_replies_df.sample(3)

,Unnamed: 0,source,text,tokens_text,text_type,date,year,month,id,conversation_id,retweet_count,reply_count,like_count,quote_count,num_tokens
109801,142664,twitter-replies,I do! I use tampons when I'm out and menstrua...,use tampons menstrual cup home suck iud lol ne...,iud,2020-09-19T03:25:14.000Z,2020,9,1307158784494546949,1307137287663357953,0,0,0,0,18
7016,8855,twitter-replies,It’s a great documentary. My ex-wife was forc...,great documentary wife forced get iud implante...,iud,2020-01-04T11:01:36.000Z,2020,1,1213415176411779073,1213211937561075713,0,0,7,0,17
41397,53595,twitter-replies,Not iud or arm rod it had me bleeding months ...,iud arm rod bleeding months weeks get cycle ma...,iud,2019-02-07T08:51:01.000Z,2019,2,1093431928236240896,1093427111967318017,0,0,0,0,13


<br><br><br><br>

# Combine into one dataframe

In [14]:
combined_df = pd.concat([reddit_posts_df, reddit_comments_df, twitter_posts_df, twitter_replies_df, webmd_df])
len(combined_df)

1063672

In [15]:
combined_df['source'].value_counts()

twitter-posts      499796
reddit-comments    264912
twitter-replies    211896
reddit-posts        68958
webmd-reviews       18110
Name: source, dtype: int64

<br><br><br><br>

# Lexicon brainstorming

In [16]:
random_sample_of_texts = []

for i, r in combined_df[combined_df['source'] == 'twitter-replies'].sample(10000).iterrows():
    random_sample_of_texts.append(' '.join(r['text'].split()))

for i, r in combined_df[combined_df['source'] == 'twitter-posts'].sample(10000).iterrows():
    random_sample_of_texts.append(' '.join(r['text'].split()))

for i, r in combined_df[combined_df['source'] == 'reddit-comments'].sample(10000).iterrows():
    random_sample_of_texts.append(' '.join(r['text'].split()))

for i, r in combined_df[combined_df['source'] == 'reddit-posts'].sample(10000).iterrows():
    random_sample_of_texts.append(' '.join(r['text'].split()))

for i, r in combined_df[combined_df['source'] == 'webmd-reviews'].sample(10000).iterrows():
    random_sample_of_texts.append(' '.join(r['text'].split()))

In [29]:
target = 'pain from my period'

matches = []
for _text in random_sample_of_texts:
    if target in _text.lower():
        matches.append(_text)
len(matches)

# category = 'nausea'

# matches = []
# for _text in random_sample_of_texts:
#     _match = False
#     for _pattern in category_patterns_dict[category]:
#         if len(re.findall(_pattern, _text.lower())) > 0:
#             _match = True
#     if _match:
#         matches.append(_text)
# len(matches)

3

In [32]:
for m in random.sample(matches, 3):
    print(m.lower().replace(target, '______' + str(target.upper()) + '______'))
    print()

# for m in random.sample(matches, 10):
#     print(m)
#     print()

when i was about 16/17 years old my periods were so bad that i couldn’t not get out of bed because of the ______PAIN FROM MY PERIOD______ the birth control pill was my only choice

this treatment has been amazing. i used to be in such ______PAIN FROM MY PERIOD______s that i would pass out, and enskyce has really impacted my overall lifestyle during my period, along with not fainting.

about 5 days ago i went to my doc to tell her i needed to stop taking my birth control marvalon, due to increasing bouts of depression and other horrible side effects. she had put me onto a new birth control pill that was less over powering so i don't feel shitty as much, however. because i got this birth control in the middle of my old pack, which i had stopped taking for 4 days, i'm unsure how to go about taking the new ones. do i start them now? do i wait till sunday? or just wait till the month is done than begin them? i absolutely cannot continue taking my old pack. the moment i stopped taking them t

<br><br><br><br>

# Define the lexicon

In [46]:
# new dictionary to better match survey paper (Nelson)

category_patterns_dict = {'breast sensitivity': [r'(breast(?:s)?|boob(?:s)?) (hurt|(is|are) (sore|sensitive|tender))',
                                                 r'soreness of (breast(?:s)?|boob(?:s)?)',
                                                 r'(tender|sore|sensitive) (breast(?:s)?|boob(?:s)?)',
                                                 r'(breast(?:s)?|boob(?:s)?) pain'],
                        'nausea': ['nausea', 
                                   'naseuous', 
                                   'sick to my stomach', 
                                   'queasy', 
                                   'vomit', 
                                   'throw up', 
                                   'puke',
                                   'puking'],
                        'skin conditions': ['skin is clearing up', 
                                            'breaking out',
                                            'broke out',
                                            'break out',
                                            'acne', 
                                            'pimples', 
                                            'melasma'], 
                        'menstrual bleeding': ['spotting',
                                               'breakthrough bleed',
                                               'bleed nonstop',
                                               'period nonstop',
                                               'nonstop period',
                                               'bled for six months',
                                               'heavy period',
                                               r'period(?:s)? (became|is|are) (regular|heavy|light|irregular)',
                                               'missed period',
                                               'missed my period'
                                               'skipped period',
                                               'skipped my period',
                                               'no period',
                                               'bleed',
                                               'heavy period',
                                               'light period'], 
                        'menstrual bleeding irregularities': ['spotting',
                                                              'breakthrough bleed',
                                                              'bleed nonstop',
                                                              'period nonstop',
                                                              'nonstop period',
                                                              'heavy period',
                                                              r'period(?:s)? (became|is|are) (heavy|light|irregular)',
                                                              'bleed',
                                                              'heavy period',
                                                              'light period'], 
                        'not having monthly period': ['missed period',
                                                      'missed my period'
                                                      'skipped period',
                                                      'skipped my period',
                                                      'no period'],
                        'weight and appetite': [r'(gain(?:ed)?|lost|lose) a (couple|few) pounds', 
                                              r'(gain(?:ed)?|lost|lose) weight',
                                              r'(gain(?:ed)?|lost|lose) [a-z0-9]+ (pounds|kilograms|lb|lbs|kg|stone)'
                                              'appetite',
                                              'craving',
                                              'hungry'],
                        'weight gain': [r'(gain(?:ed)?) a (couple|few) pounds', 
                                        r'(gain(?:ed)?) weight',
                                        r'(gain(?:ed)?) [a-z0-9]+ (pounds|kilograms|lb|lbs|kg|stone)'],
                        'mood changes': ['anxiety', 
                                         'mood', 
                                         'depression', 
                                         'depressed', 
                                         'anxious', 
                                         'anxiety',
                                         'mental health', 
                                         'panic attack', 
                                         'irritable', 
                                         'irritability',
                                         'emotional'], 
                        'libido': ['sex drive', 
                                   'sexual drive',
                                   'libido'],
                        'bloating': ['bloat',
                                     'water retention'],
                        'fatigue': ['sleepy', 
                                    'can\'t sleep', 
                                    'tired all the time', 
                                    'always feel tired', 
                                    'exhausted',
                                    'fatigue', 
                                    'always tired'], 
                        'pms': ['pms'],
                        'hair loss': ['hair loss',
                                      'hair fell',
                                      'hair thinning',
                                      'bald',
                                      'hair fall',
                                      'hair shed'],
                        'headache and migraine': ['headache', 
                                                  'head hurt',
                                                  'head ache',
                                                  'migraine'],
                        'headache': ['headache', 
                                     'head hurt',
                                     'head ache'],
                        'infection': ['infection', 
                                      'uti'], # careful, need to search for token not any string containing (maybe do this for any target with fewer than X characters)
                        'severe effects': ['blood clot', 
                                           'pulmonary embolism', 
                                           'stroke',
                                           'heart attack'],
                        'blood clot and stroke': ['blood clot', 
                                                  'pulmonary embolism', 
                                                  'stroke'],
                        'vaginal discharge': ['discharge'],
                        'dryness': ['lubricat', 
                                    'dryness', 
                                    'wet'],
                        'cramps': ['cramp', 
                                   'period pain', 
                                   'painful period', 
                                   'pain from my period'],
                        'pain': ['painful', 
                                 'pain', 
                                 'hurt',
                                 'agony',
                                 'cramp',
                                 'throb',
                                 'stabbing',
                                 'stabbed',
                                 'ache'], # will include headache but that's ok? should we also include 'tender', 'sensitive' which will include breast sensitivity?
                        'partner felt strings': [r'(partner|husband|boyfriend|bf|he|they) (could feel|felt) the strings'],
                        'general side effects': ['side effect']}

In [35]:
# category_patterns_dict = {'breast sensitivity': [r'(breast(?:s)?|boob(?:s)?) (hurt|(is|are) (sore|sensitive|tender))',
#                                                  r'soreness of (breast(?:s)?|boob(?:s)?)',
#                                                  r'(tender|sore|sensitive) (breast(?:s)?|boob(?:s)?)',
#                                                  r'(breast(?:s)?|boob(?:s)?) pain'],
#                         'nausea': ['nausea', 
#                                    'naseuous', 
#                                    'sick to my stomach', 
#                                    'queasy', 
#                                    'vomit', 
#                                    'throw up', 
#                                    'puke',
#                                    'puking'],
#                         'skin conditions': ['skin is clearing up', 
#                                             'breaking out',
#                                             'broke out',
#                                             'break out',
#                                             'acne', 
#                                             'pimples', 
#                                             'melasma'], 
#                         'menstrual bleeding': ['spotting',
#                                                'breakthrough bleed',
#                                                'bleed nonstop',
#                                                'period nonstop',
#                                                'nonstop period',
#                                                'bled for six months',
#                                                'heavy period',
#                                                r'period(?:s)? (became|is|are) (regular|heavy|light|irregular)',
#                                                'missed period',
#                                                'missed my period'
#                                                'skipped period',
#                                                'skipped my period',
#                                                'no period',
#                                                'bleed',
#                                                'heavy period',
#                                                'light period'], 
#                         'weight & appetite': [r'(gain(?:ed)?|lost|lose) a (couple|few) pounds', 
#                                               r'(gain(?:ed)?|lost|lose) weight',
#                                               r'(gain(?:ed)?|lost|lose) [a-z0-9]+ (pounds|kilograms|lb|lbs|kg|stone)'
#                                               'appetite',
#                                               'craving',
#                                               'hungry'],
#                         'mental health': ['anxiety', 
#                                           'mood', 
#                                           'depression', 
#                                           'depressed', 
#                                           'anxious', 
#                                           'anxiety',
#                                           'mental health', 
#                                           'panic attack', 
#                                           'irritable', 
#                                           'irritability',
#                                           'emotional'], 
#                         'sex drive': ['sex drive', 
#                                       'sexual drive',
#                                       'libido'],
#                         'bloating': ['bloat',
#                                      'water retention'],
#                         'sleep': ['sleepy', 
#                                   'can\'t sleep', 
#                                   'tired all the time', 
#                                   'always feel tired', 
#                                   'exhausted',
#                                   'fatigue', 
#                                   'always tired'], 
#                         'pms': ['pms'],
#                         'hair loss': ['hair loss',
#                                       'hair fell',
#                                       'hair thinning',
#                                       'bald',
#                                       'hair fall',
#                                       'hair shed'],
#                         'headache': ['headache', 
#                                      'head hurt',
#                                      'head ache'],
#                         'migraine': ['migraine'],
#                         'infection': ['infection', 
#                                       'uti'], # careful, need to search for token not any string containing (maybe do this for any target with fewer than X characters)
#                         'severe effects': ['blood clot', 
#                                            'pulmonary embolism', 
#                                            'stroke',
#                                            'heart attack'],
#                         'vaginal discharge': ['discharge'],
#                         'dryness': ['lubricat', 'dryness', 'wet'],
#                         'pain': ['painful', 
#                                  'pain', 
#                                  'hurt',
#                                  'agony',
#                                  'cramp',
#                                  'throb',
#                                  'stabbing',
#                                  'stabbed',
#                                  'ache'], # will include headache but that's ok? should we also include 'tender', 'sensitive' which will include breast sensitivity?
#                         'partner felt strings': [r'(partner|husband|boyfriend|bf|he|they) (could feel|felt) the strings'],
#                         'general side effects': ['side effect']}

<br><br><br><br>

# Find and save all the matches

In [47]:
for _category, _patterns in category_patterns_dict.items():

    # if _category == 'breast sensitivity':

    print(str(datetime.now()) + ' ' + _category)

    _matched_ids = []

    for i, r in combined_df.iterrows():

        _text = str(r['text']).lower()
        if not pd.isnull(r['title']):
            _text += ' ' + str(r['title'])

        _match = False
        for _pattern in _patterns:
            if len(_pattern) >= 4:
                if len(re.findall(_pattern, _text.lower())) > 0:
                    _match = True
            else:
                if _pattern in _text.lower().replace('.', ' ').replace('!', ' ').replace('?', ' ' ).replace(',', ' ').replace(';', ' ').split():
                    _match = True
        if _match:
            _matched_ids.append(r['id'])

    print(str(datetime.now()) + ' ' + str(len(_matched_ids)))

    _output_file = open(output_directory_path + '/lexicon-matches/' + '_'.join(_category.split()) + '.txt', 'w')
    for _id in _matched_ids:
        _output_file.write(str(_id) + '\n')
    _output_file.close()

2022-09-12 12:25:11.072106 breast sensitivity


<br><br><br><br>

# Print a sample of matches for examination

In [44]:
target_side_effect = 'blood clot and stroke'

target_ids = [l.strip() for l in open(output_directory_path + '/lexicon-matches/' + '_'.join(target_side_effect.split()) + '.txt', 'r')]
target_df = combined_df[combined_df['id'].isin(target_ids)]

print(len(target_ids))

for i, r in target_df.sample(20).iterrows():
    _text = ''
    if not pd.isnull(r['title']):
        _text += r['title']
    _text += ' ' + ' '.join(r['text'].split())
    _text = _text.lower()
    for _pattern in category_patterns_dict[target_side_effect]:
        _text = re.sub(_pattern, '----------' + _pattern + '----------', _text)
    _text
    print(_text)
    print()

# made me dry
# get dry, got dry
# vaginal dryness
# dryness?

9089
 i was late 20s, married, no kids and i wanted to find a more permanent birth control method. so i started researching what the procedure "tying your tubes" entailed / read a ton of experiences after my paragard partially expelled at one year. i decided on a tubal ligation with filshie clips. it has the best effective rate i could find (lifetime average effective rate of 99.75% see: http://www.ncbi.nlm.nih.gov/pubmed/16259814 or http://www.ncbi.nlm.nih.gov/m/pubmed/24560346/?i=1&amp;amp;amp;amp;from=filshie%20clips) so i scheduled an appointment with my obgyn, she is on the younger side and very openminded (there is a link in the sidebar to open minded doctors if you don't have a open minded obgyn). we had a 30 minute conversation and she agreed that i was well researched / she was comfortable i understood it was permanent. after the consult, i discussed with her scheduling nurse when i would prefer to schedule it (it was around 3 months out). cost: during my research, i triple ch

<br><br><br><br>

# Create CSV for labeling

In [38]:
category_matches_dict = defaultdict(list)
for _file_name in os.listdir(output_directory_path + '/lexicon-matches'):
    if _file_name.endswith('.txt'):
        _category = ' '.join(_file_name.replace('.txt', '').split('_')).strip()
        print(_file_name)
        for _line in open(output_directory_path + '/lexicon-matches/' + _file_name, 'r'):
            if _line.strip():
                category_matches_dict[_category].append(_line.strip())
len(category_matches_dict)

libido.txt
headache_and_migraine.txt
partner_felt_strings.txt
general_side_effects.txt
blood_clot_and_stroke.txt
pms.txt
not_having_monthly_period.txt
dryness.txt
cramps.txt
bloating.txt
sleep.txt
migraine.txt
weight_gain.txt
pain.txt
weight_&_appetite.txt
sex_drive.txt
skin_conditions.txt
headache.txt
hair_loss.txt
mood_changes.txt
fatigue.txt
mental_health.txt
nausea.txt
vaginal_discharge.txt
menstrual_bleeding.txt
infection.txt
menstrual_bleeding_irregularities.txt
breast_sensitivity.txt
severe_effects.txt


29

In [39]:
id_data_dict = {}
for i, r in combined_df.iterrows():
    # id_text_dict[str(r['id'])] = ' '.join(r['text'].split())
    # if not pd.isnull(r['title']):
    #     id_text_dict[str(r['id'])] += '\n' + r['title']
    id_data_dict[str(r['id'])] = r
len(id_data_dict)

1063672

In [40]:
dicts_to_label = []
for _category, _matches in category_matches_dict.items():
    for _id in random.sample(_matches, 10):
        _text = ' '.join(id_data_dict[_id]['text'].split())
        if not pd.isnull(id_data_dict[_id]['title']):
            _text = ' '.join(str(id_data_dict[_id]['title']).split()) + ' ' + _text
        dicts_to_label.append({'Side Effect': _category,
                               'Matched': 'yes',
                               'Source': id_data_dict[_id]['source'],
                               'ID': str(_id),
                               'Text': _text})
len(dicts_to_label)

290

In [53]:
all_ids = random.sample(list(id_data_dict.keys()), 10000)
for _category, _matches in category_matches_dict.items():
    print(_category)
    _unmatched_ids = [_id for _id in all_ids if _id not in _matches]
    for _id in random.sample(_unmatched_ids, 10):
        _text = ' '.join(id_data_dict[_id]['text'].split())
        if not pd.isnull(id_data_dict[_id]['title']):
            _text = ' '.join(str(id_data_dict[_id]['title']).split()) + ' ' + _text
        dicts_to_label.append({'Side Effect': _category,
                               'Matched': 'no',
                               'Source': id_data_dict[_id]['source'],
                               'ID': str(_id),
                               'Text': _text})
len(dicts_to_label)

bloating
breast sensitivity
dryness
general side effects
hair loss
headache
infection
menstrual bleeding
mental health
migraine
nausea
pain
partner felt strings
pms
severe effects
sex drive
skin conditions
sleep
vaginal discharge
weight & appetite


400

In [54]:
df_to_label = pd.DataFrame(dicts_to_label)
df_to_label.sample(5)

,Side Effect,Matched,Source,ID,Text
111,pain,yes,twitter-posts,903408502911115266,Hi I just got my old IUD removed and a new one...
39,general side effects,yes,twitter-posts,822339655693729792,"Really tho, should I get an IUD? What are the ..."
61,infection,yes,twitter-replies,127003196618047488,"""can't stds crawl up the SPINE of the IUD and ..."
207,bloating,no,twitter-posts,650464160371843076,I'm playing WordBrain and I've just reached br...
136,pms,yes,webmd-reviews,w5245,Im into my second pack 1st month period came a...


In [55]:
df_to_label.to_csv(output_directory_path + '/lexicon_matches_to_label.csv')